In [1]:
#import packages
import csv
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
from joblib import dump, load

In [24]:
#data normalization
def normalize(data):
    data_ = []
    for i in range(len(data)):
        row = []
        for j in range(len(data[i])):
            if j == 2:
                row.append(int(data[i][j])/12)
            elif j == 3:
                val = (float(data[i][j]) +60)/60
                row.append(val)
            elif j == 10:
                row.append(int(round(float(data[i][j])))/200)
            else:
                row.append(float(data[i][j]))
        data_.append(row)
    return data_

In [25]:
#load centroids from step2 clustering results
def getCentroids():
    cs = []
    with open("centroids_2.csv", "r") as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            drow = []
            for col in row:
                drow.append(float(col))
            cs.append(drow)
    #cs = np.array(cs)
    return cs

In [26]:
#每個sample使用features如下，共11個。
#['danceability','energy','key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness',
# 'liveness','valence','tempo']
#classify a feature vector. x must be a 1x11 vector(11 features)
def classify(centroids,x):
    minDis = 0
    belong = 0
    for i in range(len(centroids)):
        val = 0
        for j in range(len(centroids[0])):
            val += math.pow(x[j]-centroids[i][j],2)
        if i == 0:
            minDis = val
        elif val < minDis:
            minDis = val
            belong = i
    return belong

In [68]:
def getDataClusters(data):
    #load pca model from step2
    pca = load("pca.joblib")
    
    #get centroids from reord
    centroids = getCentroids()
    clusterAmount = len(centroids)
    #noramlize data
    data = normalize(data)
    
    #transform data with pca model
    data = np.array(data, dtype=float)
    data = pca.transform(data)
    
    #cluster stats
    nums = []
    for i in range(clusterAmount):
        nums.append(0)
    
    #classify every sample in data and output nums of each class
    clus = []
    for i in range(len(data)):
        label = classify(centroids, data[i])
        clus.append(label)
        nums[label]+=1
        
    return clus, nums

In [69]:
#load data
test_x = []
#set dataset path:
path = "../playlistresult.csv"
#header = []
with open(path, "r") as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        test_x.append(row[:11])

In [70]:
test_y, test_y_stats = getDataClusters(test_x)

In [71]:
print(test_y[:10])
print(test_y_stats)

[4, 9, 3, 4, 8, 7, 9, 8, 0, 0]
[13, 7, 2, 8, 8, 3, 5, 5, 4, 7, 11, 8, 5, 4, 10]
